In [2]:
import numpy as np
import os

In [2]:
def list_files(dir, key):
    """
    This function walks through a directory and makes a list of the files that have a name containing a particular string
    :dir: path to the directory to explore
    :key: string to look for in file names
    :return: list of files containing "key" in their filename
    """

    r = []  # List of files to be joined together
    subdirs = [x[0] for x in os.walk(dir)]
    for subdir in subdirs:
        files = next(os.walk(subdir))[2]

        for file in files:
            isTrajectory = file.find(key)
            if isTrajectory >= 0:
                r.append(subdir + "/" + file)
    return r

In [3]:
def extract_molpro(MolproInput):
    """
    This function takes one Molpro .out file and returns the geometry, the energy and the partial charges on the atoms.
    :MolproInput: the molpro .out file (string)
    :return:
    :rawData: List of strings with atom label and atom coordinates - example ['C', '0.1, '0.1', '0.1', ...]
    :ene: Value of the energy (string)
    :partialCh: List of strings with atom label and its partial charge - example ['C', '6.36', 'H', ...]
    """

    # This is the input file
    inputFile = open(MolproInput, 'r')

    # This will contain the data
    rawData = []
    ene = "0"
    partialCh = []

    for line in inputFile:
        # The geometry is found on the line after the keyword "geometry={"
        if "geometry={" in line:
            for i in range(7):
                line = next(inputFile)
                line = line.strip()
                lineSplit = line.split(" ")
                for j in range(len(lineSplit)):
                    rawData.append(lineSplit[j])
        # The energy is found two lines after the keyword "Final beta  occupancy:"
        elif "!RHF-UCCSD(T)-F12b energy" in line:
            line = line.strip()
            ene = line[len("!RHF-UCCSD(T)-F12b energy"):].strip()
        elif "Total charge composition:" in line:
            line = next(inputFile)
            line = next(inputFile)
            for i in range(7):
                line = next(inputFile)
                lineSplit = line.rstrip().split(" ")
                lineSplit = list(filter(None, lineSplit))
                partialCh.append(lineSplit[1])
                partialCh.append(lineSplit[-2])

    return rawData, ene, partialCh

In [4]:
file_list = list_files("/Volumes/Transcend/data_sets/ccsd", "_uCCSD_avtz.out")[:5]
file_list.sort()

In [5]:
# Iterating over all the files
full_geom = []
for item in file_list:
    # Extracting the geometry and the energy from a Molpro out file
    geom, ene, partial_ch = extract_molpro(item)

    if len(geom) != 28 or ene == "0" or len(partial_ch) != 14:
        print("The following file couldn't be read properly:" + str(item) + "\n")

    full_geom.append(geom)

In [6]:
xyz = []
Zs = []

element_to_Zs = {"C": 6, "H":1, "N":7}

In [7]:
for item in full_geom:
    xyz_sample = []
    Zs_sample = []
    for i in range(0,28,4):
        Zs_sample.append(element_to_Zs[item[i]])
        xyz_float = list(map(float, item[i+1:i+4]))
        xyz_sample.append(xyz_float)
    xyz.append(xyz_sample)
    Zs.append(Zs_sample)

In [8]:
xyz_np = np.asarray(xyz)
Zs_np = np.asarray(Zs)

In [9]:
Zs_np.shape
xyz_np.shape

(5, 7, 3)

In [10]:
element_list = np.array([7, 6, 1])
pair_list = np.array([[1, 1], [6, 1], [7, 1], [6, 6], [6, 7]])

In [12]:
outfile = "data_test_acsf.npz"
np.savez(outfile, xyz_np, Zs_np, element_list, pair_list)

In [17]:
input_data = "data_test_acsf.npz"
data = np.load(input_data)
data.files

['arr_0', 'arr_1', 'arr_2', 'arr_3']

In [3]:
xyzs = np.array([[[0.0, 0.0, 0.0],
                      [1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0]]])

Zs_list = np.array([[7, 2, 1, 1]])

elements_list = np.array([1, 2, 7])
element_pairs_list = np.array([[1, 1], [2, 1], [7, 1], [7, 2]])

In [4]:
outfile = "data_test_acsf_01.npz"
np.savez(outfile, xyzs, Zs_list, elements_list, element_pairs_list)

In [5]:
xyzs = np.array([[[0.0, 0.0, 0.0],
                      [1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0] ], [[0.0, 0.0, 0.0],
                      [1.0, 0.0, 0.0],
                      [0.0, 1.0, 0.0],
                      [0.0, 0.0, 1.0]] ])

Zs_list = np.array([[7, 2, 1, 1], [7, 2, 1, 1]])

elements_list = np.array([1, 2, 7])
element_pairs_list = np.array([[1, 1], [2, 1], [7, 1], [7, 2]])

In [6]:
outfile = "data_test_acsf_02.npz"
np.savez(outfile, xyzs, Zs_list, elements_list, element_pairs_list)